In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
#Reading the data

X = pd.read_csv('train.csv',index_col = 'id')
X_test_full = pd.read_csv('test.csv', index_col = 'id')

In [5]:
#Seeing what the data looks like 

X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Making sure there are no null values in the columns


cols_with_missing_vals = [col for col in X.columns if X[col].isnull().any()]

print(cols_with_missing_vals)

[]


In [7]:


#Dropping target with Null values and removing the target from the X variable

X.dropna(axis=0, subset=['target'], inplace=True)
y = X.target              
X.drop(['target'], axis=1, inplace=True)





In [8]:
#Breaking off the validation data from the training data

from sklearn.model_selection import train_test_split

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)


In [9]:
#Finding all the object columns with their number of unique values and storing the values in 

d = {}

for col in X_train_full.columns:
    if X_train_full[col].dtype == "object":
        d[col] = X_train_full[col].nunique()


for k,v in d.items():
    print(k,v)


    
obj_cols = []
# getting all the columns names into  a list
for k,v in d.items():
    obj_cols.append(k)
    
    


cat0 2
cat1 2
cat2 2
cat3 4
cat4 4
cat5 4
cat6 8
cat7 8
cat8 7
cat9 15


In [10]:
#Getting numeric columns

numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

print(numeric_cols)

for i in range(0,len(numeric_cols)):
    print(numeric_cols[i], X_train_full[numeric_cols[i]].nunique())




['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']
cont0 239766
cont1 239826
cont2 239844
cont3 239621
cont4 239822
cont5 239851
cont6 239837
cont7 239799
cont8 239790
cont9 239780
cont10 239745
cont11 239832
cont12 239780
cont13 239813


In [11]:
X_train = X_train_full.copy()
X_valid = X_valid_full.copy()
X_test = X_test_full.copy()


In [12]:
#one hot encoding the data

X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [13]:
X_train.shape

(240000, 70)

In [14]:
#Tring hyper paramerter optimization

n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]
base_score=[0.25,0.5,0.75,1]

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import xgboost

regressor=xgboost.XGBRegressor()


random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=10,
            scoring = 'neg_mean_absolute_error',n_jobs = 6,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [18]:
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:  5.2min
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed: 35.7min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed: 35.7min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n...
                                          tree_method=None,
                                          validate_parameters=None,
                                          verbosity=None),
   

In [19]:
random_cv.best_estimator_

XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=1100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
#trying to predict values using XG Boost



from sklearn.metrics import mean_absolute_error


regressor = XGBRegressor(base_score=0.75, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=4, missing=None, monotone_constraints='()',
             n_estimators=1100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)


regressor.fit(X_train, y_train, 
             early_stopping_rounds=7, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)


# Get predictions
predictions = regressor.predict(X_valid) # Your code here

# Calculate MAE
mae = mean_absolute_error(predictions, y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae)


Mean Absolute Error: 0.5741652776636893


In [23]:
#getting predections from the test data 

pred_test = regressor.predict(X_test)



# Save test predictions to file
output = pd.DataFrame({'id': X_test.index,
                       'target': pred_test})
output.to_csv('submission.csv', index=False)

